# Lab 1 - Overview of embeddings-based retrieval

Welcome! Here's a few notes about the Chroma course notebooks.
 - A number of warnings pop up when running the notebooks. These are normal and can be ignored.
 - Some operations such as calling an LLM or an opeation using generated data return unpredictable results and so your notebook outputs may differ from the video.
  
Enjoy the course!

In [1]:
from rich import print

In [ ]:
%cd ..
%pwd  # To verify the current working directory

In [ ]:
from pypdf import PdfReader

reader = PdfReader("eval/microsoft_annual_report_2022.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

print(pdf_texts[0])

In [ ]:
# This notebook is in the eval folder.  Change to the root folder.
%cd ..
%pwd  # To verify the current working directory

In [ ]:
# Load docs
# --a->: Read in the markdown files in the Obsidian vault directory
from src.ingest_service import IngestService
from src.doc_stats import DocStats
# The Directory containing the knowledge documents used by the AI to do the analysis on the soil tests.
soil_knowledge_directory = r"G:\My Drive\Audios_To_Knowledge\knowledge\AskGrowBuddy\AskGrowBuddy\Knowledge\soil_test_knowlege"
# Load the documents
ingest_service = IngestService()
loaded_documents = ingest_service.load_obsidian_notes(soil_knowledge_directory)
# Show some summary stats about the documents

DocStats.print_llama_index_docs_summary_stats(loaded_documents)

In [5]:
from rich import print

You can view the pdf in your browser [here](./microsoft_annual_report_2022.pdf) if you would like. 

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter


In [ ]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print(character_split_texts[10])
print(f"\nTotal chunks: {len(character_split_texts)}")

In [ ]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(token_split_texts[10])
print(f"\nTotal chunks: {len(token_split_texts)}")

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_split_texts[10]]))

In [11]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("microsoft_annual_report_2022", embedding_function=embedding_function, metadata={"hnsw:space": "cosine"})

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

349

In [12]:
query = "What was the total revenue?"

results = chroma_collection.query(query_texts=[query], n_results=3,
include=['documents', 'distances'])
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(document)
    print('\n')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

revenue, classified by significant product and service offerings, was as follows : ( in millions ) year ended june 
30, 2022 2021 2020 server products and cloud services $ 67, 321 $ 52, 589 $ 41, 379 office products and cloud 
services 44, 862 39, 872 35, 316 windows 24, 761 22, 488 21, 510 gaming 16, 230 15, 370 11, 575 linkedin 13, 816 
10, 289 8, 077 search and news advertising 11, 591 9, 267 8, 524 enterprise services 7, 407 6, 943 6, 409 devices 
6, 991 6, 791 6, 457 other 5, 291 4, 479 3, 768 total $ 198, 270 $ 168, 088 $ 143, 015 we have recast certain 
previously reported amounts in the table above to conform to the way we internally manage and monitor our business.

74 note 13 — unearned revenue unearned revenue by segment was as follows : ( in millions ) june 30, 2022 2021 
productivity and business processes $ 24, 558 $ 22, 120 intelligent cloud 19, 371 17, 710 more personal computing 
4, 479 4, 311 total $ 48, 408 $ 44, 141 changes in unearned revenue were as follows : ( in millions ) year ended 
june 30, 2022 balance, beginning of period $ 44, 141 deferral of revenue 110, 455 recognition of unearned revenue (
106, 188 ) balance, end of period $ 48, 408 revenue allocated to remaining performance obligations, which includes 
unearned revenue and amounts that will be invoiced and recognized as revenue in future periods, was $ 193 billion 
as of june 30, 2022, of which $ 189 billion is related to the commercial portion of revenue. we expect to recognize
approximately 45 % of this revenue over the next 12

that are not sold separately. • we tested the mathematical accuracy of management ’ s calculations of revenue and 
the associated timing of revenue recognized in the financial statements.

In [13]:
retrieved_documents[0]

'revenue, classified by significant product and service offerings, was as follows : ( in millions ) year ended june 30, 2022 2021 2020 server products and cloud services $ 67, 321 $ 52, 589 $ 41, 379 office products and cloud services 44, 862 39, 872 35, 316 windows 24, 761 22, 488 21, 510 gaming 16, 230 15, 370 11, 575 linkedin 13, 816 10, 289 8, 077 search and news advertising 11, 591 9, 267 8, 524 enterprise services 7, 407 6, 943 6, 409 devices 6, 991 6, 791 6, 457 other 5, 291 4, 479 3, 768 total $ 198, 270 $ 168, 088 $ 143, 015 we have recast certain previously reported amounts in the table above to conform to the way we internally manage and monitor our business.'

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer

# Assuming you've already initialized your embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # or whichever model you're using

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

query = "What was the total revenue?"

# Get the query embedding
query_embedding = model.encode([query])[0]

# Perform the query and get both documents and embeddings
results = chroma_collection.query(
    query_texts=[query],
    n_results=3,
    include=['documents', 'embeddings']
)

retrieved_documents = results['documents'][0]
retrieved_embeddings = results['embeddings'][0]

# Calculate cosine similarities
similarities = [cosine_similarity(query_embedding, doc_embedding)
                for doc_embedding in retrieved_embeddings]

# Print documents with their cosine similarities
for document, similarity in zip(retrieved_documents, similarities):
    print(f"Document: {document}")
    print(f"Cosine Similarity: {similarity:.4f}")
    print()

In [ ]:
import os
import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()

In [ ]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report."
            "You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
output = rag(query=query, retrieved_documents=retrieved_documents)

print(word_wrap(output))